Decision Question 1

Where are functioning health centres consistently low — and is that changing?

This analysis focuses on identifying districts that show persistently low reported health facility functionality over time, rather than short-term shocks or single-month anomalies.

Several safeguards are applied to avoid over-interpretation:

Analysis is conducted at the district–month level.

Only districts with sufficient reporting history are included.

Periods of reporting collapse are excluded from trend calculations.

Median values are used to summarise functionality, as they are more robust to outliers and reporting noise than means.

This approach prioritises signal stability and interpretability over completeness.

In [77]:
# We first identify districts with persistently low health facility functionality.
# To avoid misleading rankings driven by sparse or late-period data:
# - We exclude months flagged as low coverage.
# - We require a minimum number of reporting months per district.
# - We summarise functionality using the median, not the mean.

import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

# -------------------------------------------------------------------
# 1) Load processed dataset (Phase 3 output)
# -------------------------------------------------------------------
df = pd.read_csv("../data_processed/yemen_health_cleaned_deduped.csv")
df["month"] = pd.to_datetime(df["month"])

df.shape
df.head()

# -------------------------------------------------------------------
# 2) Coverage diagnostics (non-negotiable for interpretation)
#    We measure how many districts report each month and flag "low coverage".
# -------------------------------------------------------------------
coverage = (
    df.groupby("month")["pcode_2"]
    .nunique()
    .reset_index(name="num_districts_reporting")
)

# Inspect distribution (used to justify the threshold choice)
coverage["num_districts_reporting"].describe()

# ACAP-style judgement rule:
# Months with fewer than this many districts reporting are treated as low-confidence.
absolute_floor = 150
coverage["low_coverage_month"] = coverage["num_districts_reporting"] < absolute_floor

# Quick check of coverage flags at start and end of time series
coverage.head()
coverage.tail()

# Merge coverage info back into the main dataset (so every output can reference it)
df = df.merge(coverage, on="month", how="left")

df[["month", "num_districts_reporting", "low_coverage_month"]].drop_duplicates().head()

# -------------------------------------------------------------------
# 3) Decision Question 1:
#    Where are functioning health centres consistently low (with enough reporting)?
# -------------------------------------------------------------------

# How many months does each district report? (used to justify min_months)
district_months = df.groupby("pcode_2")["month"].nunique()
district_months.describe()

# Minimum reporting requirement:
# Districts must report functionality data for at least 12 months
# to be considered in the sustained-functionality ranking.
min_months = 12

# Exclude low coverage months and missing functionality values before ranking.
func_df = df[
    (df["low_coverage_month"] == False) &
    (df["percent_functioning_health_centres"].notna())
].copy()

# Rank districts by median functionality (robust metric for messy data)
func_rank = (
    func_df
    .groupby(["pcode_2", "district", "governorate"], as_index=False)
    .agg(
        months_reporting=("month", "nunique"),
        median_functioning=("percent_functioning_health_centres", "median")
    )
)

# Apply the reporting sufficiency rule
func_rank = func_rank[func_rank["months_reporting"] >= min_months]

# Lowest median functionality = highest concern signal
func_rank = func_rank.sort_values("median_functioning")

# Show the worst 20 (dashboard-ready decision table)
func_rank.head(20)

# To assess whether functionality is changing over time,
# we compare median functionality between two stable reporting periods.
# This is a descriptive comparison only and does not imply causality.

early_period = df[
    (df["month"] < "2020-01-01") &
    (df["low_coverage_month"] == False) &
    (df["percent_functioning_health_centres"].notna())
]

late_period = df[
    (df["month"] >= "2020-01-01") &
    (df["month"] < "2021-01-01") &
    (df["low_coverage_month"] == False) &
    (df["percent_functioning_health_centres"].notna())
]

# Period selection:
# - Early period: 2019
# - Later period: 2020
# 2021 is excluded due to widespread reporting collapse.

early_func = (
    early_period
    .groupby(["pcode_2", "district", "governorate"], as_index=False)
    .agg(
        early_months=("month", "nunique"),
        early_median=("percent_functioning_health_centres", "median")
    )
)

late_func = (
    late_period
    .groupby(["pcode_2", "district", "governorate"], as_index=False)
    .agg(
        late_months=("month", "nunique"),
        late_median=("percent_functioning_health_centres", "median")
    )
)
change_df = early_func.merge(
    late_func,
    on=["pcode_2", "district", "governorate"],
    how="inner"
)

# Require at least 6 months in each period  to consider the change reliable
change_df = change_df[
    (change_df["early_months"] >= 6) &
    (change_df["late_months"] >= 6)
]

change_df["delta_functioning"] = (
    change_df["late_median"] - change_df["early_median"]
)

change_df.sort_values("delta_functioning").head(20)


,pcode_2,district,governorate,early_months,early_median,late_months,late_median,delta_functioning


Decision Question 2

Which governorates and districts show sustained cholera pressure?

This analysis aims to identify districts experiencing persistent cholera pressure, rather than short-lived outbreaks or reporting artefacts.

Cholera pressure is assessed using:

reported suspected cholera cases (counts),

persistence over time (number of months with elevated case counts),

and sufficient reporting coverage.

Given known limitations in death and attack rate reporting, suspected case counts are treated as the primary signal, with other indicators used cautiously and descriptively.

This analysis is intended to support prioritisation and investigation, not burden estimation.

That framing is very ACAP-appropriate.

In [78]:
# -------------------------------------------------------------------
# Prepare dataset for cholera pressure analysis
# -------------------------------------------------------------------
# We focus on suspected cholera cases, which have the highest completeness.
# Low coverage months are excluded to avoid artificial drops or spikes.

chol_df = df[
    (df["low_coverage_month"] == False) &
    (df["num_suspected_cases_cholera"].notna())
].copy()

chol_df.shape

# Define a high cholera month threshold (top 10% of reported cases)
high_case_threshold = chol_df["num_suspected_cases_cholera"].quantile(0.90)

high_case_threshold

# -------------------------------------------------------------------
# District-level cholera pressure summary
# -------------------------------------------------------------------
chol_pressure = (
    chol_df
    .groupby(["pcode_2", "district", "governorate"], as_index=False)
    .agg(
        months_reporting=("month", "nunique"),
        total_suspected_cases=("num_suspected_cases_cholera", "sum"),
        high_case_months=(
            "num_suspected_cases_cholera",
            lambda x: (x >= high_case_threshold).sum()
        )
    )
)

# Require a minimum reporting history to avoid one-off spikes
chol_pressure = chol_pressure[chol_pressure["months_reporting"] >= 12]

# Rank by persistence first, then by total burden
chol_pressure = chol_pressure.sort_values(
    ["high_case_months", "total_suspected_cases"],
    ascending=False
)

chol_pressure.head(20)

# Governorate-level summary for high-level prioritisation
chol_gov = (
    chol_pressure
    .groupby("governorate", as_index=False)
    .agg(
        districts_affected=("pcode_2", "nunique"),
        total_cases=("total_suspected_cases", "sum"),
        total_high_case_months=("high_case_months", "sum")
    )
    .sort_values("total_high_case_months", ascending=False)
)

chol_gov


,governorate,districts_affected,total_cases,total_high_case_months
15,Sana'a,16,140887.0,126
11,Ibb,20,99624.0,81
9,Dhamar,12,83773.0,74
10,Hajjah,29,94199.0,61
7,Al Mahwit,9,43509.0,44
3,Al Bayda,19,65010.0,37
8,Amran,20,61722.0,36
4,Al Hodeidah,16,34179.0,27
18,Ta'iz,17,25155.0,11
16,Sana'a City,7,17285.0,8


Districts in and around Sana’a, as well as selected urban centres in Hajjah, Dhamar, and Ibb, show sustained cholera pressure between 2019 and 2020, characterised by repeated months of elevated suspected case counts. The persistence of these signals, combined with consistent reporting coverage, suggests chronic transmission risk rather than isolated outbreaks.

Decision Question 3

Do drops in reported health facility functionality align with spikes in suspected cholera?

This analysis explores whether districts show systematic co-movement between reported health facility functionality and suspected cholera cases over time.

The objective is signal detection, not causal inference. Any observed relationships may reflect:

operational constraints,

surveillance artefacts,

population movement,

or contextual factors outside the health system.

Analysis is therefore conducted:

within districts (to avoid cross-district confounding),

only where both indicators are reported for a sufficient number of months,

and excluding periods of reporting collapse.

In [79]:
# -------------------------------------------------------------------
# Prepare paired functionality–cholera dataset
# -------------------------------------------------------------------
# We retain only months where:
# - coverage is adequate
# - both functionality and suspected cholera cases are reported

pair_df = df[
    (df["low_coverage_month"] == False) &
    (df["percent_functioning_health_centres"].notna()) &
    (df["num_suspected_cases_cholera"].notna())
].copy()

pair_df.shape

min_ts_months = 8

# -------------------------------------------------------------------
# Compute within-district correlations
# -------------------------------------------------------------------
def corr_if_sufficient(g):
    # Require enough temporal variation
    if g["month"].nunique() < min_ts_months:
        return np.nan
    
    return g["percent_functioning_health_centres"].corr(
        g["num_suspected_cases_cholera"]
    )

corr_df = (
    pair_df
    .groupby(["pcode_2", "district", "governorate"], as_index=False)
    .apply(lambda g: pd.Series({
        "months_used": g["month"].nunique(),
        "corr_func_vs_cholera": corr_if_sufficient(g)
    }))
)

# Drop districts where correlation could not be computed
corr_df = corr_df.dropna(subset=["corr_func_vs_cholera"])

# Sort by strongest negative correlations first
corr_df = corr_df.sort_values("corr_func_vs_cholera")

corr_df.head(20)


c:\Users\ChikaAnanaba\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\ChikaAnanaba\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\ChikaAnanaba\AppData\Local\Temp\ipykernel_23916\361780124.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,pcode_2,district,governorate,months_used,corr_func_vs_cholera
135,YE2314,Bani Dabyan,Sana'a,12.0,-0.636566
126,YE2305,Sanhan wa Bani Bahlul,Sana'a,12.0,-0.558289
42,YE1412,Radman,Al Bayda,12.0,-0.543813
57,YE1608,Al Ghayl,Al Jawf,12.0,-0.543749
46,YE1417,Sabah,Al Bayda,12.0,-0.512570
51,YE1602,Al Humaydat,Al Jawf,12.0,-0.504901
39,YE1409,Al Bayda City,Al Bayda,12.0,-0.492153
55,YE1606,Al Mutun,Al Jawf,12.0,-0.477035
127,YE2306,Bilad Ar Rus,Sana'a,12.0,-0.473688
40,YE1410,Al Bayda,Al Bayda,12.0,-0.440795


Districts with strong negative correlations indicate periods where lower reported health facility functionality tends to coincide with higher suspected cholera case counts.
These patterns should be interpreted as signals warranting further investigation, not evidence of causation.

In [80]:
## -------------------------------------------------------------------
# Decision Q3: Within-district correlation (signal scan, not causality)
# Future-proof: no groupby.apply + avoids invalid correlations
# -------------------------------------------------------------------

pair_df = df[
    (df["low_coverage_month"] == False) &
    (df["percent_functioning_health_centres"].notna()) &
    (df["num_suspected_cases_cholera"].notna())
].copy()

min_ts_months = 8

def safe_corr(a, b):
    a = pd.Series(a)
    b = pd.Series(b)

    if len(a) < min_ts_months:
        return np.nan
    if a.nunique() < 2 or b.nunique() < 2:
        return np.nan

    return a.corr(b)

corr_df = (
    pair_df
    .groupby(["pcode_2", "district", "governorate"], as_index=False)
    .agg(
        months_used=("month", "nunique"),
        corr_func_vs_cholera=("num_suspected_cases_cholera",
                              lambda x: safe_corr(
                                  pair_df.loc[x.index, "percent_functioning_health_centres"],
                                  x
                              ))
    )
    .dropna(subset=["corr_func_vs_cholera"])
)

corr_df["strong_negative_signal"] = corr_df["corr_func_vs_cholera"] <= -0.5
corr_df = corr_df.sort_values("corr_func_vs_cholera")
corr_df.head(10)


,pcode_2,district,governorate,months_used,corr_func_vs_cholera,strong_negative_signal
135,YE2314,Bani Dabyan,Sana'a,12,-0.636566,True
126,YE2305,Sanhan wa Bani Bahlul,Sana'a,12,-0.558289,True
42,YE1412,Radman,Al Bayda,12,-0.543813,True
57,YE1608,Al Ghayl,Al Jawf,12,-0.543749,True
46,YE1417,Sabah,Al Bayda,12,-0.512570,True
51,YE1602,Al Humaydat,Al Jawf,12,-0.504901,True
39,YE1409,Al Bayda City,Al Bayda,12,-0.492153,False
55,YE1606,Al Mutun,Al Jawf,12,-0.477035,False
127,YE2306,Bilad Ar Rus,Sana'a,12,-0.473688,False
40,YE1410,Al Bayda,Al Bayda,12,-0.440795,False


Districts flagged with strong negative correlations show periods where declining reported health facility functionality coincides with increasing suspected cholera cases.

These patterns should be interpreted as signals warranting further investigation, rather than evidence of a direct causal relationship. Possible explanations include overlapping operational constraints, population movement, surveillance effects, or contextual factors beyond the health system.

Synthesis: Converging decision signals

This table brings together three analytical signals:

Persistently low health facility functionality

Sustained cholera pressure

Strong negative co-movement between functionality and cholera

The purpose is to identify districts where multiple independent signals converge, suggesting areas that may warrant prioritised attention or deeper investigation.

Inclusion in this table does not imply causality or programme performance assessment. Signals reflect reported data patterns under variable coverage and reporting constraints.

In [81]:
# -------------------------------------------------------------------
# Signal 1: Persistently low functionality
# -------------------------------------------------------------------
# Flag districts in the lowest-functioning group (from func_rank)

low_func_flag = func_rank.copy()
low_func_flag["low_functionality_signal"] = True

low_func_flag = low_func_flag[
    ["pcode_2", "district", "governorate", "low_functionality_signal"]
]

# -------------------------------------------------------------------
# Signal 2: Sustained cholera pressure
# -------------------------------------------------------------------
chol_flag = chol_pressure.copy()

chol_flag["sustained_cholera_pressure"] = (
    chol_flag["high_case_months"] >= 10
)

chol_flag = chol_flag[
    ["pcode_2", "district", "governorate", "sustained_cholera_pressure"]
]

# -------------------------------------------------------------------
# Synthesis: merge three decision signals at district level
# -------------------------------------------------------------------

# Threshold used for "sustained cholera pressure"
HIGH_CASE_MONTHS_THRESHOLD = 10

keys = ["pcode_2", "district", "governorate"]
signal_cols = [
    "low_functionality_signal",
    "sustained_cholera_pressure",
    "strong_negative_signal"
]

# Base: unique district list from the working analysis dataframe
synthesis = df[keys].drop_duplicates()

# --- Signal 1: Persistently low functionality (districts included in func_rank)
low_func_flag = (
    func_rank[keys]
    .drop_duplicates()
    .assign(low_functionality_signal=True)
)

# --- Signal 2: Sustained cholera pressure (based on high_case_months threshold)
chol_flag = (
    chol_pressure[keys + ["high_case_months"]]
    .drop_duplicates()
    .assign(sustained_cholera_pressure=lambda d: d["high_case_months"] >= HIGH_CASE_MONTHS_THRESHOLD)
    .loc[:, keys + ["sustained_cholera_pressure"]]
)

# --- Signal 3: Strong negative co-movement (already computed)
corr_flag = (
    corr_df[keys + ["strong_negative_signal"]]
    .drop_duplicates()
)

# Merge signals
synthesis = (
    synthesis
    .merge(low_func_flag, on=keys, how="left")
    .merge(chol_flag, on=keys, how="left")
    .merge(corr_flag, on=keys, how="left")
)

# Ensure all flags are explicit boolean values
for c in signal_cols:
    synthesis[c] = synthesis[c].astype("boolean").fillna(False)

# Count how many signals each district exhibits
synthesis["num_signals"] = synthesis[signal_cols].sum(axis=1)

# Sort by convergence (most signals first)
synthesis = synthesis.sort_values(
    ["num_signals", "governorate", "district"],
    ascending=[False, True, True]
)

# Actionable shortlist: districts with 2+ converging signals
priority_districts = synthesis[synthesis["num_signals"] >= 2].copy()

# Governorate-level summary
synthesis_gov = (
    synthesis
    .groupby("governorate", as_index=False)
    .agg(
        districts_with_signals=("num_signals", lambda x: (x >= 2).sum()),
        max_signals=("num_signals", "max")
    )
    .sort_values(["districts_with_signals", "max_signals"], ascending=False)
)

synthesis.head(20), priority_districts.head(20), synthesis_gov

keys = ["pcode_2", "district", "governorate"]
signal_cols = ["low_functionality_signal", "sustained_cholera_pressure", "strong_negative_signal"]

priority_view = (
    synthesis[synthesis["num_signals"] >= 2]
    .loc[:, keys + signal_cols + ["num_signals"]]
    .sort_values(["num_signals", "governorate", "district"], ascending=[False, True, True])
)

print(priority_view.head(30).to_string(index=False))

synthesis.to_csv("../data_processed/synthesis_district_signals.csv", index=False)
priority_view.to_csv("../data_processed/synthesis_priority_districts.csv", index=False)
synthesis_gov.to_csv("../data_processed/synthesis_governorate_summary.csv", index=False)






pcode_2                district governorate  low_functionality_signal  sustained_cholera_pressure  strong_negative_signal  num_signals
 YE2305   Sanhan wa Bani Bahlul      Sana'a                      True                        True                    True            3
 YE1412                  Radman    Al Bayda                      True                       False                    True            2
 YE1417                   Sabah    Al Bayda                      True                       False                    True            2
 YE1608                Al Ghayl     Al Jawf                      True                       False                    True            2
 YE1602             Al Humaydat     Al Jawf                      True                       False                    True            2
 YE2915                   Amran       Amran                      True                        True                   False            2
 YE1728             Hajjah City      Hajjah            

In [82]:
import pandas as pd

df_syn = pd.read_csv("../data_processed/synthesis_district_signals.csv")

df_syn.head(10)
df_syn.dtypes
df_syn["num_signals"].value_counts().sort_index()
df_syn.dtypes
df_syn[df_syn["num_signals"] == 3]

pd.read_csv("../data_processed/synthesis_priority_districts.csv").head()
pd.read_csv("../data_processed/synthesis_governorate_summary.csv")



,governorate,districts_with_signals,max_signals
0,Sana'a,8,3
1,Ibb,4,2
2,Al Bayda,2,2
3,Al Jawf,2,2
4,Amran,1,2
5,Hajjah,1,2
6,Abyan,0,1
7,Aden,0,1
8,Dhamar,0,1
9,Lahj,0,1
